In [1]:
#  Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [2]:
# loadmat aux function to load the matlab save files
# from: https://stackoverflow.com/review/suggested-edits/21667510

import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _has_struct(elem):
        """Determine if elem is an array and if any array item is a struct"""
        return isinstance(elem, np.ndarray) and any(isinstance(
                    e, scipy.io.matlab.mio5_params.mat_struct) for e in elem)

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif _has_struct(elem):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif _has_struct(sub_elem):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [3]:
# Matlab code conversion

# Read matlab file instead of excelfile (easier conversion)
data = loadmat('matlab.mat')

C:\Users\camar\AppData\Local\Temp\ipykernel_2660\3560610281.py:19: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
C:\Users\camar\AppData\Local\Temp\ipykernel_2660\3560610281.py:35: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):
C:\Users\camar\AppData\Local\Temp\ipykernel_2660\3560610281.py:26: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  e, scipy.io.matlab.mio5_params.mat_struct) for e in elem)


In [4]:
# Linearize the generator prices

# genCofA=data.generator.limit(:,:,6); -> MATLAB starts with 1, so we need the index -1
# genCofB=data.generator.limit(:,:,7);
# genCofC=data.generator.limit(:,:,8);

genCofA = data['data']['generator']['limit'][:, :, 5]
genCofB = data['data']['generator']['limit'][:, :, 6]
genCofC = data['data']['generator']['limit'][:, :, 7]

In [5]:
# Options for simulation

#options 
iterlim = 1000000000
reslim = 5000000000

In [6]:
# Definir o numero de resources usados

numGen = np.arange(1, data['data']['parameterData']['resources']['numGen'] + 1)
numLoad = np.arange(1, data['data']['parameterData']['resources']['numLoad'] + 1)
numStor = np.arange(1, data['data']['parameterData']['resources']['numStor'] + 1)
numV2G = np.arange(1, data['data']['parameterData']['resources']['numV2G'] + 1)
numCStat = np.arange(1, data['data']['parameterData']['resources']['numCStat'] + 1)
numPeriod = np.arange(1, data['data']['parameterData']['resources']['period'] + 1)
numBus = np.arange(1) # forced to 1
nOwner = np.arange(1, data['data']['parameterData']['resources']['owners'] + 1)

In [7]:
# Define as strings com os recursos para mandar para GAMS

gen = np.max(numGen)
Load = np.max(numLoad)
stor = np.max(numStor)
v2g = np.max(numV2G)
cs = np.max(numCStat)
period = np.max(numPeriod)
Bus = np.max(numBus)

In [8]:
#% Definir as matrizes com os dados para o GAMS

# General Info
pMaxImp = data['data']['parameterData']['generalInfo']['P_Max_Imp']
pMaxExp = data['data']['parameterData']['generalInfo']['P_Max_Exp']
buyPrice = data['data']['parameterData']['generalInfo']['Energy_Buy_Price']
sellPrice = data['data']['parameterData']['generalInfo']['Energy_Sell_Price']

# Geradores
genLimit = data['data']['generator']['limit'][numGen[0]-1:numGen[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
genInfo = data['data']['generator']['info']

# Cargas
loadLimit = data['data']['load']['limit'][numLoad[0]-1:numLoad[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]

# Baterias
storLimit = data['data']['storage']['limit'][numStor[0]-1:numStor[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
storInfo = data['data']['storage']['info']

# Veiculos
v2gLimit = data['data']['vehicle']['limit']
v2gInfo = data['data']['vehicle']['info']

# Charging Station
csLimit = data['data']['cstation']['limit'][numCStat[0]-1:numCStat[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
csInfo = data['data']['cstation']['info']

# Connect EV to Charging Station
EV_CS_Info = data['data']['vehicle']['timeInfo']['V2GinCS']

In [9]:
# Definir os dados da rede para o GAMS

# Linhas
branchData = data['data']['network']['branch']

# Tensão Máxima
voltageMax = data['data']['parameterData']['network']['voltageMax']

# Tensão Mínima
voltageMin = data['data']['parameterData']['network']['voltageMin']

# Angulo Máximo
angleMax = data['data']['parameterData']['network']['angleMax']

# Angulo Mínimo
angleMin = data['data']['parameterData']['network']['angleMin']

# Definir a matriz das admitancias
ybus = data['data']['parameterData']['network']['ybus']
diag = data['data']['parameterData']['network']['diag']
branchID = data['data']['parameterData']['network']['branch']

In [10]:
#% Anular o custo fixo e quadratico da DG

genLimit[:, :, 3] = 0
genLimit[:, :, 5] = 0

genLimit = np.append(genLimit, np.zeros(shape=(7, 24, 4)), axis=2)

# Not necessary anymore
#genLimit[:, :, 9] = 0
#genLimit[:, :, 11] = 0

In [11]:
# TODO -> AFTER GAM FILE CONVERSION

def runSolver(file, loadLimit, genLimit, genInfo, gen, Load, period, stor, storLimit, storInfo,
              pMaxImp, pMaxExp, buyPrice, sellPrice, v2gLimit, v2gInfo, v2g,
              cs, cslimit, csInfo, EV_CS_Info):
    
    
    temp_output = [resFun, resCost, resSelf, statusOptPro, 
                   resPImp, resPExp, resGenAct, resGenExcAct,resGenX, 
                   resLoadRed, resLoadCut, 
                   resLoadENS, resLoadX, resLoad, resStorEnerState, resStorChActPower, resStorDchActPower,
                   resv2gEnerState, resv2gDchActPower, resv2gChActPower, resv2gRelax, resCsPower]
    
    return temp_output

In [12]:
#** Define the fixed sets structure to use in the parameters of the gams optimization process
#set genLimitInfo 'Indicate the number of columns of information from generators'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set loadLimitInfo 'Indicate the number of columns of information from loads'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set storLimitInfo 'Indicate the number of columns of information from storage units'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set v2gLimitInfo 'Indicate the number of columns of information from electric vehicles'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

#set csLimitInfo 'Indicate the number of columns of information from charging stations'
#/1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12/;

NCOLS = 12

genLimitInfo = np.arange(1, NCOLS+1)
loadLimitInfo = np.arange(1, NCOLS+1)
storLimitInfo = np.arange(1, NCOLS+1)
v2gLimitInfo = np.arange(1, NCOLS+1)
csLimitInfo = np.arange(1, NCOLS+1)

In [13]:
# * Define the several sets structure used in the gams optimization process, 
# these sets depends the number of resources used in matlab code

t = np.arange(1, period+1)
gen = np.arange(1, gen+1)
Load = np.arange(1, Load+1)
stor = np.arange(1, stor+1)
v2g = np.arange(1, v2g+1)
cs = np.arange(1, cs+1)

In [14]:
# Pyomo imports

import pyomo
import pyomo.opt
import pyomo.environ as pe

In [312]:
# Model creation

model = pe.ConcreteModel()

model.genLimitInfo = pe.Set(initialize=genLimitInfo, 
                            doc='Indicate the number of columns of information from generators')
model.loadLimitInfo = pe.Set(initialize=loadLimitInfo, 
                             doc='Indicate the number of columns of information from loads')
model.storLimitInfo = pe.Set(initialize=storLimitInfo, 
                             doc='Indicate the number of columns of information from storage units')
model.v2gLimitInfo = pe.Set(initialize=v2gLimitInfo, 
                            doc='Indicate the number of columns of information from electric vehicles')
model.csLimitInfo = pe.Set(initialize=csLimitInfo, 
                           doc='Indicate the number of columns of information from charging stations')

# Sets
model.t = pe.Set(initialize=t, doc='period')
model.gen = pe.Set(initialize=gen, doc='generators')
model.loads = pe.Set(initialize=Load, doc='loads')
model.stor = pe.Set(initialize=stor, doc='storage')
model.v2g = pe.Set(initialize=v2g, doc='vehicles')
model.cs = pe.Set(initialize=cs, doc='charging_stations')

In [313]:
# Define the parameters with the data from the excel, such as limits, price, and voltage

model.loadLimit = pe.Param(model.loads, model.t, initialize=model.loadLimitInfo)
model.genLimit = pe.Param(model.gen, model.t, initialize=model.genLimitInfo)
model.genInfo = pe.Param(model.gen, initialize=model.genLimitInfo)
model.pMaxImp = pe.Param(model.t, initialize=0)
model.buyPrice = pe.Param(model.t, initialize=0)
model.pMaxExp = pe.Param(model.t, initialize=0)
model.sellPrice = pe.Param(model.t, initialize=0)
model.storLimit = pe.Param(model.stor, model.t, initialize=model.storLimitInfo)
model.v2gLimit = pe.Param(model.v2g, model.t, initialize=model.v2gLimitInfo)
model.v2gInfo = pe.Param(model.v2g, initialize=model.v2gLimitInfo)
model.csLimit = pe.Param(model.cs, model.t, initialize=model.csLimitInfo)
model.csInfo = pe.Param(model.cs, initialize=model.csLimitInfo)
model.EV_CS_Info = pe.Param(model.v2g, model.cs, model.t)

In [314]:
# parameters  loadActPower(load,t), optCost;

model.loadActPower = pe.Param(model.loads, model.t, initialize=1)
model.optCost = pe.Param()

In [315]:
# loadActPower(load,t)=loadLimit(load,t,'1'); -> NEED TO ASK!!!

In [316]:
# Define the variables for the gams optimization process
# variables fun, derCost, selfConso;

model.fun = pe.Var()
model.derCost = pe.Var()
model.selfConso = pe.Var()

In [317]:
#positive variables

model.genActPower = pe.Var(model.gen, model.t, initialize=0, bounds=(0.0, None))
model.genExcActPower = pe.Var(model.gen, model.t, initialize=0, bounds=(0.0, None))
model.loadRedActPower = pe.Var(model.loads, model.t, initialize=0, bounds=(0.0, None))
model.loadCutActPower = pe.Var(model.loads, model.t, initialize=0, bounds=(0.0, None))
model.loadENS = pe.Var(model.loads, model.t,initialize=0,  bounds=(0.0, None))
model.pImp = pe.Var(model.t, initialize=0, bounds=(0.0, None))
model.pExp = pe.Var(model.t, initialize=0, bounds=(0.0, None))
model.storEnerState = pe.Var(model.stor, model.t, initialize=0, bounds=(0.0, None))
model.storDchActPower = pe.Var(model.stor, model.t, initialize=0, bounds=(0.0, None))
model.storChActPower = pe.Var(model.stor, model.t, initialize=0, bounds=(0.0, None))
model.EminRelaxStor = pe.Var(model.stor, model.t, initialize=0, bounds=(0.0, None))
model.v2gDchActPower = pe.Var(model.v2g, model.t, initialize=0, bounds=(0.0, None))
model.v2gChActPower = pe.Var(model.v2g, model.t, initialize=0, bounds=(0.0, None))
model.v2gEnerState = pe.Var(model.v2g, model.t, initialize=0, bounds=(0.0, None))
model.EminRelaxEv = pe.Var(model.v2g, model.t, initialize=0, bounds=(0.0, None))
model.csActPower = pe.Var(model.cs, model.t, initialize=0, bounds=(0.0, None))
model.csActPowerNet = pe.Var(model.cs, model.t, initialize=0, bounds=(0.0, None))

In [318]:
# binary variables

model.genXo = pe.Var(model.gen, model.t, initialize=0, bounds=(0, 1))
model.loadXo = pe.Var(model.loads, model.t, initialize=0, bounds=(0, 1))
model.v2gChXo = pe.Var(model.v2g, model.t, initialize=0, bounds=(0, 1))
model.v2gDchXo = pe.Var(model.v2g, model.t, initialize=0, bounds=(0, 1))
model.storChXo = pe.Var(model.stor, model.t, initialize=0, bounds=(0, 1))
model.storDchXo = pe.Var(model.stor, model.t, initialize=0, bounds=(0, 1))

### Cost Minimization Objective Cost Function
optCostFun.. derCost=e=  sum((gen,t), genActPower(gen,t)*genLimit(gen,t,'3')+genExcActPower(gen,t)*genLimit(gen,t,'5'))
                       + sum((load,t), loadRedActPower(load,t)*loadLimit(load,t,'7')+loadCutActPower(load,t)*loadLimit(load,t,'8')+loadENS(load,t)*loadLimit(load,t,'10'))
                       + sum((stor,t), storDchActPower(stor,t)*storLimit(stor,t,'4')-storChActPower(stor,t)*storLimit(stor,t,'3') + EminRelaxStor(stor,t)*200)
                       + sum((v2g,t), v2gDchActPower(v2g,t)*v2gLimit(v2g,t,'7')-v2gChActPower(v2g,t)*v2gLimit(v2g,t,'6') + EminRelaxEv(v2g,t)*200)
                       + sum(t,PImp(t)*buyPrice(t)-PExp(t)*sellPrice(t))
                       ;

###  Self-Consuption Maximization Objective Cost Function
optSelfFun.. selfConso=e=sum(t,PImp(t))
                       + sum((gen,t), 10000*genExcActPower(gen,t))+ sum((load,t), 10000*loadENS(load,t))
                       + sum((stor,t),1000*EminRelaxStor(stor,t)) + sum((v2g,t), 1000*EminRelaxEv(v2g,t))
                       + sum((load,t),10*(loadRedActPower(load,t) + loadCutActPower(load,t)))
                       + sum(t,PExp(t)*100*(sum ((stor),(storLimit(stor,t,'1')-storChActPower(stor,t))) + sum((cs), csInfo(cs,'5')-csActPower(cs,t))))
                       + sum(t,PExp(t)*500*(sum ((stor),(storDchActPower(stor,t)))))
                       + 0.0001*derCost
                       ;

In [319]:
# Network constraints

def _maxImpEq(m, s):
    return m.pImp[s] <= m.pMaxImp[s] 

model.MaxImpEq = pe.Constraint(model.t, rule=_maxImpEq)

def _maxExpEq(m, s):
    return m.pExp[s] <= m.pMaxExp[s]

model.MaxExpEq = pe.Constraint(model.t, rule=_maxExpEq)

In [ ]:
# Generator constraints with the active generation power
# Maximum generation in generators with normal contract

#genActMaxEq1(gen,t)$(genInfo(gen,'5')=1).. genActPower(gen,t)=e=genLimit(gen,t,'1');
#*Minimum generation in generators with normal contract
#genActMinEq(gen,t)$(genInfo(gen,'5')=1).. genActPower(gen,t)=g=genInfo(gen,'7')*genXo(gen,t);
#*Generation in generators with Feed-in tariffs
#genActMaxEq2(gen,t)$(genInfo(gen,'5')=2).. genActPower(gen,t)+genExcActPower(gen,t)=e=genLimit(gen,t,'1');


# IMPORTANT: DOLLAR SIGN IS A TERNARY IF OPERATOR
# EXAMPLE: a $ (b > 1.5) = 2 -> if b > 1.5 then a=2

#def _genActMaxEq1(m, s):
#    m.genActPower[s] == m.genLimit[] 